In [9]:
from os.path import join

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import sentencepiece as spm
from sklearn.model_selection import train_test_split

In [2]:
np.random.seed(42)

In [3]:
DATA_DIR = join('..', 'data', 'raw')

In [4]:
train_full_df = pd.read_csv(join(DATA_DIR, 'train.tsv'), sep='\t')
train_full_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0
1,402555,536040,536041,How do I control my horny emotions?,How do you control your horniness?,1
2,360472,364011,490273,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0
3,150662,155721,7256,What can one do after MBBS?,What do i do after my MBBS ?,1
4,183004,279958,279959,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0


In [5]:
train_append_df = pd.read_csv(join(DATA_DIR, 'test.tsv'), sep='\t')
test_df = pd.read_csv(join(DATA_DIR, 'dev.tsv'), sep='\t')

In [7]:
train_full_df = pd.concat([train_full_df, train_append_df])
train_full_df = train_full_df[['question1', 'question2']]

In [10]:
train_df, dev_df = train_test_split(train_full_df, test_size=0.1, random_state=42)

In [11]:
with open('../data/processed/train_samples.txt', 'w') as f:
    for _, row in train_df.iterrows():
        f.write(row['question1'] + '\n')
        f.write(row['question2'] + '\n')

In [19]:
spm.SentencePieceTrainer.train(
    input='../data/processed/train_samples.txt', model_prefix='../models/trained/spm-8k', vocab_size=8000)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../data/processed/train_samples.txt
  input_format: 
  model_prefix: ../models/trained/spm-8k
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy:

In [22]:
sp = spm.SentencePieceProcessor(model_file='../models/trained/spm-8k.model', add_bos=True, add_eos=True)

In [23]:
text = "This is a sample sentence."
sp.encode(text, out_type=int)

[1, 4259, 9, 8, 5024, 539, 22, 2]

In [25]:
sp.decode([1, 4259, 9, 8, 5024, 539, 22, 2])

'This is a sample sentence.'